In [2]:
!pip install rapidfuzz yfinance pandas numpy nltk scikit-learn


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\Crook\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
from rapidfuzz import process, fuzz
import yfinance as yf
import pandas as pd
import numpy as np
import nltk
import re

import json

In [90]:
# LOAD SEC(https://www.sec.gov/files/company_tickers.json) TICKERS
with open("tickers.json", "r") as f:
    tickers = json.load(f)
    companies = [tickers[i]["title"] for i in tickers]
    abbrs = [tickers[i]["ticker"] for i in tickers]

    comp_tckr = {comp: abbr for comp, abbr in zip(companies, abbrs)}

# LOAD INITIAL RECALL DATA
df = pd.read_csv("recalls.csv")

## Data Cleaning

In [91]:
# delete instances where year is older than 1995
print(len(df))
for i, row in df.iterrows():
    if ( row['Report Received Date'][-4:].isnumeric() and int(row['Report Received Date'][-4:]) < 1995 ):
        df.drop(i, inplace=True)
print(len(df))

29374
22505


In [92]:
# remove column that won't feed into training
df = df.drop("Completion Rate % (Blank - Not Reported)", axis=1)

In [95]:
# from first version - not using this in this version
df = df.drop("Opening Stock Value", axis =1)
df = df.drop("Closing Stock Value", axis =1)

In [96]:
import re
from collections import Counter

# Common words to ignore
stop_words = set([
    "inc", "corp", "llc", "co", "ltd", "corporation",
    "company", "limited", "plc", "gmbh", "sa", "ag", "pte", "bv", "kg", "kgaa", "corp", "industries", "us", "of"
])

def tokenize(name):
    name = name.lower()
    name = re.sub(r'[^a-z0-9 ]', ' ', name) 
    words = name.split()
    return [w for w in words if w not in stop_words]

# Build a word-to-company mapping for fast lookup
word_counts = Counter()
company_word_map = {}
for c in companies:
    words = tokenize(c)
    company_word_map[c] = set(words)
    word_counts.update(words)


def match_by_unique_words(name):
    words = tokenize(name)
    if not words:
        return None
    
    # Score companies by sum of inverse word frequencies (rare words are more valuable)
    scores = {}
    for c, c_words in company_word_map.items():
        score = sum(1 / word_counts[w] for w in words if w in c_words)
        if score > 0:
            scores[c] = score
    
    if scores:
        return max(scores, key=scores.get)
    return None

df.loc[:2000, "fuzzy_company"] = df.loc[:2000, "Manufacturer"].apply(match_by_unique_words)

In [97]:
# get stock ticker
df["ticker"] = df["fuzzy_company"].map(comp_tckr)

# convert to datetime
df["Report Received Date"] = pd.to_datetime(df["Report Received Date"])

# get unique tickers
unique_tickers = df["ticker"].astype(str).unique().tolist()

# get all prices in range (start of day, end of day)
start_date = df["Report Received Date"].min()
end_date = df["Report Received Date"].max() + pd.Timedelta(days=1)
price_data = yf.download(unique_tickers, start=start_date, end=end_date, group_by="ticker", progress=False)

# helper function to grab pricing data
def get_price(ticker, date):
    try:
        day_data = price_data[ticker].loc[date.strftime("%Y-%m-%d")]
        return pd.Series([day_data["Open"], day_data["Close"]])
    except Exception:
        return pd.Series([None, None])

# get open/close price
df[["open", "close"]] = df.apply(lambda x: get_price(x["ticker"], x["Report Received Date"]), axis=1)

# grab info on ticker
info_cache = {}
for ticker in unique_tickers:
    try:
        t = yf.Ticker(ticker)
        info = t.info
        info_cache[ticker] = {
            "market_cap": info.get("marketCap"),
            "eps": info.get("trailingEps"),
            "full_name": info.get("longName"),
        }
    except Exception:
        pass

# merge info back on ticker
info_df = pd.DataFrame(info_cache).T
info_df.index.name = "ticker"
df = df.merge(info_df, on="ticker", how="left")


C:\Users\Crook\AppData\Local\Temp\ipykernel_10356\3663560923.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  price_data = yf.download(unique_tickers, start=start_date, end=end_date, group_by="ticker", progress=False)

15 Failed downloads:
['FFAIW', 'AMBP-WT', 'AMBI-WT', 'TGE-WT', 'F-PD', 'DSYWW']: YFPricesMissingError('possibly delisted; no price data found  (1d 1995-01-01 00:00:00 -> 2025-10-02 00:00:00)')
['EPDU', 'PLSAY', 'LBRJ', 'TDWD', 'TMRD', 'BBAAY', 'LOMWF', 'RYPBF', 'CLSO']: YFTzMissingError('possibly delisted; no timezone found')
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: TMRD"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: CLSO"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: TDWD"}}}


In [ ]:
df

,Report Received Date,NHTSA ID,Recall Link,Manufacturer,Subject,Component,Mfr Campaign Number,Recall Type,Potentially Affected,Recall Description,...,Stock Abbreviation,Opening Stock Value,Closing Stock Value,fuzzy_company,ticker,open,close,market_cap,eps,full_name
0,2025-10-01,25V656000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,Toyota Motor Engineering & Manufacturing,Driveshaft May Deform and Break,POWER TRAIN,25TB11 / 25TA11,Vehicle,5960,Toyota Motor Engineering & Manufacturing (Toyo...,...,NYSE:TM,192.39,191.83,TOYOTA MOTOR CORP/,TOYOF,17.84000,19.379999,269150797824,2.14,Toyota Motor Corporation
1,2025-10-01,25V655000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,"Nissan North America, Inc.",Fire Risk from Quick Charging Battery,ELECTRICAL SYSTEM,R25C8,Vehicle,19077,"Nissan North America, Inc. (Nissan) is recalli...",...,NYSE:NSANY,NaN,NaN,Central North Airport Group,GAERF,13.94000,13.940000,4924503552,0.73,"Grupo Aeroportuario del Centro Norte, S.A.B. d..."
2,2025-09-30,25V654000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,"Volvo Car USA, LLC",Power Operated Tailgate May Drop Suddenly,STRUCTURE,R10342,Vehicle,1119,"Volvo Car USA, LLC (Volvo) is recalling certai...",...,NYSE:,NaN,NaN,"Mister Car Wash, Inc.",MCW,5.30000,5.330000,1646194432,0.26,"Mister Car Wash, Inc."
3,2025-09-30,25V653000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,Thor Motor Coach,LP Tank May Detach and Become a Road Hazard,EQUIPMENT,RC000331,Vehicle,23,Thor Motor Coach (TMC) is recalling certain 20...,...,NYSE:,NaN,NaN,THOR INDUSTRIES INC,THO,104.54486,103.191399,5831233024,4.84,"THOR Industries, Inc."
4,2025-09-26,25E062000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,"MEDIX SPECIALTY VEHICLES, INC.",Quick Liner Wheel Covers May Detach and Become...,WHEELS,NaN,Equipment,396,"Medix Specialty Vehicles, Inc. (Medix) is reca...",...,NYSE:,NaN,NaN,"Envirotech Vehicles, Inc.",EVTV,2.02000,1.890000,5754790,-10.19,"Envirotech Vehicles, Inc."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22500,1995-01-11,95V004000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,"THOMAS BUILT BUSES, INC.",INTERIOR SYSTEMS:ACTIVE RESTRAINTS:BELT RETRAC...,SEAT BELTS,NR (Not Reported),Vehicle,3076,THE METAL HOUSING OF THE SAFETY BELT RETRACTOR...,...,NYSE:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22501,1995-01-03,95T001000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,Cooper Tire & Rubber Co.,TIRES:MARKINGS,TIRES,NR (Not Reported),Tire,6603,THE MAXIMUM INFLATION PRESSURE STAMPING ON THE...,...,NYSE:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22502,1995-01-03,95V013000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,"PREVOST CAR, INC. Inactive",STRUCTURE:DOOR ASSEMBLY,STRUCTURE,NR (Not Reported),Vehicle,1090,IF A PASSENGER TRIES TO EXIT THE LAVATORY COMP...,...,NYSE:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22503,1995-01-03,95V016000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,"PREVOST CAR, INC. Inactive",EMERGENCY PARKING BRAKE:MECHANICAL,PARKING BRAKE,NR (Not Reported),Vehicle,2075,THE PARKING BRAKE BUTTON CAN BE HIT ACCIDENTLY...,...,NYSE:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
def ba_rm_nulls(df=None, col=None):
    """ 
    Helper function to print length of dataframe before removing nulls in a column, then print the length after the nulls are removed.

    Args:
        * df (pd.DataFrame): Dataframe with nulls.
        * col (str): Column name nulls should be removed from.

    Returns:
        * df (pd.DataFrame): Dataframe without nulls.
    """

    print(f"BEFORE NULLS FROM {col} REMOVED: ", len(df))

    df = df[df[f"{col}"].notna()]

    print(f"AFTER NULLS FROM {col} REMOVED: ", len(df))

    return df

In [99]:
df = ba_rm_nulls(df, "ticker")

BEFORE NULLS FROM ticker REMOVED:  22505
AFTER NULLS FROM ticker REMOVED:  1530


In [100]:
# intermediate step to save compute time during development
df.to_csv('intermediate.csv')

In [4]:
df = pd.read_csv("intermediate.csv")

### Get Dummies

In [5]:
print(len(df.value_counts("Recall Type")))
print(len(df.value_counts("Component")))

4
32


In [6]:
df['Original_Component'] = df['Component'].fillna('')

In [7]:
df = pd.get_dummies(data=df, prefix="RECALL_TYPE_", columns=["Recall Type"])
df = pd.get_dummies(data=df, prefix="COMPONENT_TYPE_", columns=["Component"])

### Bag of Words Featuring 

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

#### Recall Description BOW

In [9]:
df["Recall Description"]

0       Toyota Motor Engineering & Manufacturing (Toyo...
1       Nissan North America, Inc. (Nissan) is recalli...
2       Volvo Car USA, LLC (Volvo) is recalling certai...
3       Thor Motor Coach (TMC) is recalling certain 20...
4       Medix Specialty Vehicles, Inc. (Medix) is reca...
                              ...                        
1525    Porsche Cars North America, Inc. (Porsche) is ...
1526    Volkswagen Group of America, Inc. (Audi) is re...
1527    Daimler Trucks North America, LLC (DTNA) is re...
1528    Honda (American Honda Motor Co.) is recalling ...
1529    Hubbell Incorporated (Delaware) is recalling c...
Name: Recall Description, Length: 1530, dtype: object

In [10]:
recall_vectorizer = CountVectorizer(
    input='content',    
    stop_words='english',
    analyzer='word',
    decode_error='ignore'
)

component_vectorizer = CountVectorizer(
    input='content',    
    stop_words='english',
    analyzer='word',
    decode_error='ignore'
)

In [11]:
print(df.columns)
len(df.columns)

Index(['Unnamed: 0', 'Report Received Date', 'NHTSA ID', 'Recall Link',
       'Manufacturer', 'Subject', 'Mfr Campaign Number',
       'Potentially Affected', 'Recall Description', 'Consequence Summary',
       'Corrective Action', 'Park Outside Advisory', 'Do Not Drive Advisory',
       'Stock Abbreviation', 'fuzzy_company', 'ticker', 'open', 'close',
       'market_cap', 'eps', 'full_name', 'Original_Component',
       'RECALL_TYPE__Child Seat', 'RECALL_TYPE__Equipment',
       'RECALL_TYPE__Tire', 'RECALL_TYPE__Vehicle', 'COMPONENT_TYPE__AIR BAGS',
       'COMPONENT_TYPE__BACK OVER PREVENTION', 'COMPONENT_TYPE__CHILD SEAT',
       'COMPONENT_TYPE__COMMUNICATION', 'COMPONENT_TYPE__ELECTRICAL SYSTEM',
       'COMPONENT_TYPE__ELECTRONIC STABILITY CONTROL (ESC)',
       'COMPONENT_TYPE__ENGINE AND ENGINE COOLING',
       'COMPONENT_TYPE__EQUIPMENT',
       'COMPONENT_TYPE__EQUIPMENT ADAPTIVE/MOBILITY',
       'COMPONENT_TYPE__EXTERIOR LIGHTING',
       'COMPONENT_TYPE__FORWARD COLLISIO

58

In [12]:
X_recall = recall_vectorizer.fit_transform(df["Recall Description"])
X_component = component_vectorizer.fit_transform(df["Original_Component"])

In [13]:
print(recall_vectorizer.vocabulary_)

{'toyota': 4186, 'motor': 2912, 'engineering': 1721, 'manufacturing': 2753, 'recalling': 3446, 'certain': 1127, '2025': 158, 'tacoma': 4081, 'hybrid': 2267, 'vehicles': 4374, 'driveshaft': 1595, 'constant': 1273, 'velocity': 4376, 'cv': 1396, 'joint': 2457, 'deform': 1477, 'break': 974, 'nissan': 3002, 'north': 3015, 'america': 721, '2021': 153, '2022': 154, 'leaf': 2587, 'equipped': 1748, 'level': 2605, 'quick': 3386, 'charging': 1146, 'port': 3261, 'lithium': 2646, 'ion': 2426, 'battery': 864, 'overheat': 3124, 'volvo': 4434, 'car': 1072, 'usa': 4322, 'llc': 2649, 'ex90': 1780, 'spindle': 3899, 'nut': 3031, 'detach': 1503, 'power': 3276, 'operated': 3083, 'tailgate': 4084, 'pot': 3271, 'drive': 1590, 'units': 4299, 'causing': 1101, 'suddenly': 4020, 'drop': 1600, 'thor': 4135, 'coach': 1205, 'tmc': 4165, '2024': 156, 'luminate': 2699, 'indigo': 2349, 'recreational': 3459, 'lp': 2682, 'tank': 4090, 'support': 4042, 'tube': 4252, 'short': 3790, 'insufficient': 2387, 'weld': 4493, 'frac

In [14]:
print(df.columns)
len(df.columns)

Index(['Unnamed: 0', 'Report Received Date', 'NHTSA ID', 'Recall Link',
       'Manufacturer', 'Subject', 'Mfr Campaign Number',
       'Potentially Affected', 'Recall Description', 'Consequence Summary',
       'Corrective Action', 'Park Outside Advisory', 'Do Not Drive Advisory',
       'Stock Abbreviation', 'fuzzy_company', 'ticker', 'open', 'close',
       'market_cap', 'eps', 'full_name', 'Original_Component',
       'RECALL_TYPE__Child Seat', 'RECALL_TYPE__Equipment',
       'RECALL_TYPE__Tire', 'RECALL_TYPE__Vehicle', 'COMPONENT_TYPE__AIR BAGS',
       'COMPONENT_TYPE__BACK OVER PREVENTION', 'COMPONENT_TYPE__CHILD SEAT',
       'COMPONENT_TYPE__COMMUNICATION', 'COMPONENT_TYPE__ELECTRICAL SYSTEM',
       'COMPONENT_TYPE__ELECTRONIC STABILITY CONTROL (ESC)',
       'COMPONENT_TYPE__ENGINE AND ENGINE COOLING',
       'COMPONENT_TYPE__EQUIPMENT',
       'COMPONENT_TYPE__EQUIPMENT ADAPTIVE/MOBILITY',
       'COMPONENT_TYPE__EXTERIOR LIGHTING',
       'COMPONENT_TYPE__FORWARD COLLISIO

58

#### Corrective Action BOW

In [15]:
df["Corrective Action"]

0       Dealers will inspect and replace both front dr...
1       Owners are advised not to use Level 3 quick ch...
2       Owners are advised not to use the POT function...
3       The remedy is currently under development. Own...
4       Medix will replace the covers, free of charge....
                              ...                        
1525    Owners are advised not to use the 220V/240V co...
1526    Owners are advised not to use the 220V/240V co...
1527    Dealers will add additional mechanical fastene...
1528    Dealers will replace the battery cable, free o...
1529    Hubbell Incorporated will replace the receptac...
Name: Corrective Action, Length: 1530, dtype: object

In [16]:
corrective_action_vectorizer = CountVectorizer(
    input='content',    
    stop_words='english',
    analyzer='word',
    decode_error='ignore'
)

In [17]:
X_corrective_action = corrective_action_vectorizer.fit_transform(df["Corrective Action"])

In [18]:
print(corrective_action_vectorizer.vocabulary_)

{'dealers': 2012, 'inspect': 2476, 'replace': 3222, 'driveshaft': 2083, 'assemblies': 1608, 'necessary': 2852, 'free': 2262, 'charge': 1825, 'owner': 2935, 'notification': 2879, 'letters': 2607, 'expected': 2162, 'mailed': 2663, 'november': 2886, '16': 159, '2025': 330, 'owners': 2936, 'contact': 1893, 'toyota': 3635, 'customer': 1979, 'service': 3410, '800': 1311, '331': 856, '4331': 963, 'numbers': 2889, 'recall': 3168, '25tb11': 766, '25ta11': 762, 'advised': 1527, 'use': 3694, 'level': 2608, 'quick': 3101, 'charging': 1826, 'remedy': 3211, 'completed': 1872, 'update': 3687, 'battery': 1655, 'software': 3460, 'interim': 2497, 'notifying': 2883, 'safety': 3334, 'risk': 3270, 'october': 2897, '24': 429, 'second': 3393, 'notice': 2878, 'sent': 3406, 'available': 1630, 'nissan': 2866, '867': 1371, '7669': 1286, 'number': 2888, 'r25c8': 3135, 'vehicle': 3717, 'identification': 2431, 'vins': 3734, 'involved': 2506, 'searchable': 3389, 'nhtsa': 2864, 'gov': 2310, 'beginning': 1667, 'pot': 

### Linear Regression Experiment

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [31]:
df = df.dropna(subset=['open', 'close'])
y = df['open'] - df['close']
X = df[['Potentially Affected']]

In [32]:
y

0      -1.539999
1       0.000000
2      -0.030000
3       1.353461
4       0.130000
          ...   
1525   -0.100000
1526   -0.026141
1527   -0.452686
1528    0.000000
1529    4.058471
Length: 1217, dtype: float64

In [33]:
X

,Potentially Affected
0,5960
1,19077
2,1119
3,23
4,396
...,...
1525,41345
1526,93196
1527,321
1528,106030


In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
print("X_train_shape:", X_train.shape)
print("X_test_shape:", X_test.shape)
print("y_train_shape:", y_train.shape)
print("y_test_shape:", y_test.shape )

X_train_shape: (973, 1)
X_test_shape: (244, 1)
y_train_shape: (973,)
y_test_shape: (244,)


In [40]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred_lin = lin_reg.predict(X_test)

print("Linear Regression Coefficent:", lin_reg.coef_)
print("Linear Regression Intercept:", lin_reg.intercept_)
print("Mean Squared Error:", mean_squared_error(y_test, y_pred_lin))

Linear Regression Coefficent: [1.19087004e-06]
Linear Regression Intercept: -0.07999539308701267
Mean Squared Error: 57.006634781202884


In [44]:
print("R2 score: {:.8f}".format(r2_score(y_test, y_pred_lin)))

R2 score: 0.00091972
